In [ ]:
!python

In [ ]:
# Copyright (c) Facebook, Inc. and its affiliates. All rights reserved.

# Dataloaders for ShapeNetCore and R2N2
This tutorial shows how to:
- Load models from ShapeNetCore and R2N2 using PyTorch3D's data loaders.
- Pass the loaded datasets to `torch.utils.data.DataLoader`.
- Render ShapeNetCore models with PyTorch3D's renderer.
- Render R2N2 models with the same orientations as the original renderings in the dataset.
- Visualize R2N2 model voxels.

## 0. Install and import modules

If `torch`, `torchvision` and `pytorch3d` are not installed, run the following cell:

In [1]:
!pip install torch torchvision
import os
import sys
import torch
if torch.__version__=='1.6.0+cu101' and sys.platform.startswith('linux'):
    !pip install pytorch3d
else:
    need_pytorch3d=False
    try:
        import pytorch3d
    except ModuleNotFoundError:
        need_pytorch3d=True
    if need_pytorch3d:
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   118    0   118    0     0    424      0 --:--:-- --:--:-- --:--:--   424
100  404k    0  404k    0     0   470k      0 --:--:-- --:--:-- --:--:-- 2466k
  Cloning https://github.com/facebookresearch/pytorch3d.git (to revision stable) to /tmp/pip-req-build-_i7o0esh
  Running command git clone -q https://github.com/facebookresearch/pytorch3d.git /tmp/pip-req-build-_i7o0esh
  Running command git checkout -q 18ce14cd31a2f1c99e16d727e1755156c360453a
     |████████████████████████████████| 645kB 12.3MB/s 
  Created wheel for pytorch3d: filename=pytorch3d-0.3.0-cp36-cp36m-linux_x86_64.whl size=17869310 sha256=0ccdd427b72c5012e95e25cef7cbee11ed505c441447544decf4ee095832ca62
  Stored in directory: /tmp/pip-ephem-wheel-cache-qn_chx9q/wheels/89/69/08/d864f516508b5d943259c6088baa150ebdd3659b5dde4e3571
  Created wheel for fvcore: filename

In [3]:
import numpy as np
import torch

from pytorch3d.datasets import (
    R2N2,
    ShapeNetCore,
    collate_batched_meshes,
    render_cubified_voxels,
)
from pytorch3d.renderer import (
    OpenGLPerspectiveCameras,
    PointLights,
    RasterizationSettings,
    TexturesVertex,
    look_at_view_transform,
)

from pytorch3d.structures import Meshes
from torch.utils.data import DataLoader

# add path for demo utils functions 
import sys
import os
sys.path.append(os.path.abspath(''))

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



If using **Google Colab**, fetch the utils file for plotting image grids:

In [4]:
!wget https://raw.githubusercontent.com/facebookresearch/pytorch3d/master/docs/tutorials/utils/plot_image_grid.py
from plot_image_grid import image_grid

--2021-02-08 06:37:17--  https://raw.githubusercontent.com/facebookresearch/pytorch3d/master/docs/tutorials/utils/plot_image_grid.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1472 (1.4K) [text/plain]
Saving to: ‘plot_image_grid.py’

plot_image_grid.py  100%[===================>]   1.44K  --.-KB/s    in 0s      

2021-02-08 06:37:17 (33.0 MB/s) - ‘plot_image_grid.py’ saved [1472/1472]



OR if running locally uncomment and run the following cell:

In [ ]:
# from utils import image_grid

In [14]:
!wget https://github.com/facebookresearch/pytorch3d/blob/master/pytorch3d/datasets/shapenet/shapenet_synset_dict_v1.json

--2021-02-08 06:46:54--  https://github.com/facebookresearch/pytorch3d/blob/master/pytorch3d/datasets/shapenet/shapenet_synset_dict_v1.json
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘shapenet_synset_dict_v1.json’

shapenet_synset_dic     [ <=>                ] 112.89K   609KB/s    in 0.2s    

2021-02-08 06:46:55 (609 KB/s) - ‘shapenet_synset_dict_v1.json’ saved [115604]



In [15]:
!cp /content/shapenet_synset_dict_v1.json /usr/local/lib/python3.6/dist-packages/pytorch3d/datasets/shapenet/

## 1. Load the datasets

If you haven't already downloaded the ShapeNetCore dataset, first do that following the instructions here: https://www.shapenet.org/. ShapeNetCore is a subset of the ShapeNet dataset. In PyTorch3D we support both version 1 (57 categories) and version 2 (55 categories).

Then modify `SHAPENET_PATH` below to you local path to the ShapeNetCore dataset folder. 

In [ ]:
# Setup
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")
    
SHAPENET_PATH_v2 = "/content/drive/MyDrive/shapenet_re_unzip/ShapeNetCore.v2"
shapenet_dataset = ShapeNetCore(SHAPENET_PATH_v2, version=2, load_textures=True)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The R2N2 dataset can be downloaded using the instructions here: http://3d-r2n2.stanford.edu/. Look at the links for `ShapeNetRendering` and `ShapeNetVox32`. The R2N2 dataset contains 13 categories that are a subset of the ShapeNetCore v.1
dataset. The R2N2 dataset also contains its own 24 renderings of each object and voxelized models.

Then modify `R2N2_PATH` and `SPLITS_PATH` below to your local R2N2 dataset folder path and splits file path respectively. Here we will load the `train` split of R2N2 and ask the voxels of each model to be returned.

In [ ]:
R2N2_PATH = ""
SPLITS_PATH = "None"
r2n2_dataset = R2N2("train", SHAPENET_PATH, R2N2_PATH, SPLITS_PATH, return_voxels=True)

We can retrieve a model by indexing into the loaded dataset. For both ShapeNetCore and R2N2, we can examine the category this model belongs to (in the form of a synset id, equivalend to wnid described in ImageNet's API: http://image-net.org/download-API), its model id, and its vertices and faces.

In [ ]:
shapenet_model = shapenet_dataset[6]
print("This model belongs to the category " + shapenet_model["synset_id"] + ".")
print("This model has model id " + shapenet_model["model_id"] + ".")
model_verts, model_faces = shapenet_model["verts"], shapenet_model["faces"]

We can use its vertices and faces to form a `Meshes` object which is a PyTorch3D datastructure for working with batched meshes.

In [ ]:
model_textures = TexturesVertex(verts_features=torch.ones_like(model_verts, device=device)[None])
shapenet_model_mesh = Meshes(
    verts=[model_verts.to(device)],   
    faces=[model_faces.to(device)],
    textures=model_textures
)

With R2N2, we can further examine R2N2's original renderings. For instance, if we would like to see the second and third views of the eleventh objects in the R2N2 dataset, we can do the following:

In [ ]:
r2n2_renderings = r2n2_dataset[10,[1,2]]
image_grid(r2n2_renderings.numpy(), rows=1, cols=2, rgb=True)

## 2. Use the datasets with `torch.utils.data.DataLoader`

Training deep learning models, usually requires passing in batches of inputs. The `torch.utils.data.DataLoader` from Pytorch helps us do this. PyTorch3D provides a function `collate_batched_meshes` to group the input meshes into a single `Meshes` object which represents the batch. The `Meshes` datastructure can then be used directly by other PyTorch3D ops which might be part of the deep learning model (e.g. `graph_conv`).

For R2N2, if all the models in the batch have the same number of views, the views, rotation matrices, translation matrices, intrinsic matrices and voxels will also be stacked into batched tensors.

**NOTE**: All models in the `val` split of R2N2 have 24 views, but there are 8 models that split their 24 views between `train` and `test` splits, in which case `collate_batched_meshes` will only be able to join the matrices, views and voxels as lists. However, this can be avoided by laoding only one view of each model by setting `return_all_views = False`.

In [ ]:
batch_size = 12
r2n2_single_view = R2N2("train", SHAPENET_PATH, R2N2_PATH, SPLITS_PATH, return_all_views=False, return_voxels=True)
r2n2_loader = DataLoader(r2n2_single_view, batch_size=batch_size, collate_fn=collate_batched_meshes)

Let's visualize all the views (one for each model) in the batch:

In [ ]:
it = iter(r2n2_loader)
r2n2_batch = next(it)
batch_renderings = r2n2_batch["images"] # (N, V, H, W, 3), and in this case V is 1.
image_grid(batch_renderings.squeeze().numpy(), rows=3, cols=4, rgb=True)

## 3. Render ShapeNetCore models with PyTorch3D's differntiable renderer

Both `ShapeNetCore` and `R2N2` dataloaders have customized `render` functions that support rendering models by specifying their model ids, categories or indices using PyTorch3D's differentiable renderer implementation.

In [59]:
# Rendering settings.
R, T = look_at_view_transform(1.0, 1.0, 90)
cameras = OpenGLPerspectiveCameras(R=R, T=T, device=device)
raster_settings = RasterizationSettings(image_size=512)
lights = PointLights(location=torch.tensor([0.0, 1.0, -2.0], device=device)[None],device=device)

First we will try to render three models by their model ids:

In [58]:
images_by_model_ids = shapenet_dataset.render(
    model_ids=[
        "13394ca47c89f91525a3aaf903a41c90",
        "14755c2ee8e693aba508f621166382b0",
        "156c4207af6d2c8f1fdc97905708b8ea",
    ],
    device=device,
    cameras=cameras,
    raster_settings=raster_settings,
    lights=lights,
)
image_grid(images_by_model_ids.cpu().numpy(), rows=1, cols=3, rgb=True)

ValueError: ignored

Suppose we would like to render the first three models in the dataset, we can render models by their indices:

In [ ]:
images_by_idxs = shapenet_dataset.render(
    idxs=list(range(3)),
    device=device,
    cameras=cameras,
    raster_settings=raster_settings,
    lights=lights,
)
image_grid(images_by_idxs.cpu().numpy(), rows=1, cols=3, rgb=True)

Alternatively, if we are not interested in any particular models but would like see random models from some specific categories, we can do that by specifying `categories` and `sample_nums`. For example, if we would like to render 2 models from the category "faucet" and 3 models from the category "chair", we can do the following:

In [ ]:
images_by_categories = shapenet_dataset.render(
    categories=["faucet", "chair"],
    sample_nums=[2, 3],
    device=device,
    cameras=cameras,
    raster_settings=raster_settings,
    lights=lights,
)
image_grid(images_by_categories.cpu().numpy(), rows=1, cols=5, rgb=True)

If we are not interested in any particular categories and just would like to render some random models from the whole dataset, we can set the number of models to be rendered in `sample_nums` and not specify any `categories`:

In [ ]:
random_model_images = shapenet_dataset.render(
    sample_nums=[3],
    device=device,
    cameras=cameras,
    raster_settings=raster_settings,
    lights=lights,
)
image_grid(random_model_images.cpu().numpy(), rows=1, cols=5, rgb=True)

## 4. Render R2N2 models with the same orientations as the original renderings in the dataset

We can render R2N2 models the same way as we rendered ShapeNetCore models above. In addition, we can also render R2N2 models with the same orientations as the original renderings in the dataset. For this we will use R2N2's customized `render` function and a different type of PyTorch3D camera called `BlenderCamera`.

In this example, we will render the seventh model with the same orientations as its second and third views. First we will retrieve R2N2's original renderings to compare with the result.

In [ ]:
original_rendering = r2n2_dataset[6,[1,2]]["images"]
image_grid(original_rendering.numpy(), rows=1, cols=2, rgb=True)

Next, we will visualize PyTorch3d's renderings:

In [ ]:
r2n2_oriented_images = r2n2_dataset.render(
    idxs=[6],
    view_idxs=[1,2],
    device=device,
    raster_settings=raster_settings,
    lights=lights,
)
image_grid(r2n2_oriented_images.cpu().numpy(), rows=1, cols=2, rgb=True)

## 5. Visualize R2N2 models' voxels

R2N2 dataloader also returns models' voxels. We can visualize them by utilizing R2N2's `render_vox_to_mesh` function. This will cubify the voxels to a Meshes object, which will then be rendered.

In this example we will visualize the tenth model in the dataset with the same orientation of its second and third views. First we will retrieve R2N2's original renderings to compare with the result.

In [ ]:
r2n2_model = r2n2_dataset[9,[1,2]]
original_rendering = r2n2_model["images"]
image_grid(original_rendering.numpy(), rows=1, cols=2, rgb=True)

Next, we will pass the voxels to `render_vox_to_mesh`:

In [ ]:
vox_render = render_cubified_voxels(r2n2_model["voxels"], device=device)
image_grid(vox_render.cpu().numpy(), rows=1, cols=2, rgb=True)

In [31]:
import pytorch3d as p3d
print(p3d.__version__)

0.3.0


In [53]:
verts, faces, aux = p3d.io.load_obj('/content/drive/MyDrive/shapenet_re_unzip/ShapeNetCore.v2/02773838/10a885f5971d9d4ce858db1dc3499392/models/model_normalized.obj')

mesh = Meshes(verts=[verts], faces=[faces.verts_idx])

In [34]:
#camera setup
R, T = p3d.renderer.look_at_view_transform(2.7, 10, 20)
cameras = p3d.renderer.OpenGLPerspectiveCameras(device=device, R=R, T=T)

In [ ]:
#the way image will be displayed
raster_settings = p3d.renderer.RasterizationSettings(
   image_size = 512,
   blur_radius = 0.0,
   faces_per_pixel = 1,
   bin_size = 0
)

In [43]:
#placing light
lights = p3d.renderer.PointLights(device=device, location=[[1.0, 1.0, -2.0]])

In [48]:
#renderer settings
renderer = p3d.renderer.MeshRenderer(
   rasterizer = p3d.renderer.MeshRasterizer(
      cameras = cameras,
      raster_settings = raster_settings
   ),
   shader = p3d.renderer.mesh.HardPhongShader(
      device = device,
      cameras = cameras,
      lights = lights
   )
)

In [49]:
#display mesh
images = renderer(mesh)
plt.figure(figsize=(10, 10))
plt.imshow(images[0, ..., :3].cpu().numpy())
plt.grid("off")
plt.axis("off")

NameError: ignored

In [39]:
import matplotlib.pyplot as plt

In [54]:
import matplotlib.pyplot as plt




verts, faces, aux = p3d.io.load_obj('/content/drive/MyDrive/shapenet_re_unzip/ShapeNetCore.v2/02773838/10a885f5971d9d4ce858db1dc3499392/models/model_normalized.obj')
mesh = Meshes(verts=[verts], faces=[faces.verts_idx])


#camera setup
R, T = p3d.renderer.look_at_view_transform(2.7, 10, 20)
cameras = p3d.renderer.OpenGLPerspectiveCameras(device=device, R=R, T=T)


#the way image will be displayed
raster_settings = p3d.renderer.RasterizationSettings(
   image_size = 512,
   blur_radius = 0.0,
   faces_per_pixel = 1,
   bin_size = 0
)


#placing light
lights = p3d.renderer.PointLights(device=device, location=[[1.0, 1.0, -2.0]])



#renderer settings
renderer = p3d.renderer.MeshRenderer(
   rasterizer = p3d.renderer.MeshRasterizer(
      cameras = cameras,
      raster_settings = raster_settings
   ),
   shader = p3d.renderer.mesh.SoftPhongShader(
      device = device,
      cameras = cameras,
      lights = lights
   )
)


#display mesh
images = renderer(mesh)
plt.figure(figsize=(10, 10))
plt.imshow(images[0, ..., :3].cpu().numpy())
plt.grid("off")
plt.axis("off")




ValueError: ignored